# **einops**

In [3]:
!pip install einops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 682.2 kB/s eta 0:00:00


In [4]:
class EinOpsRearrange(nn.Module):
    def __init__(self, rearrange_expr: str, **kwargs) -> None:
        super().__init__()
        self.rearrange_expr = rearrange_expr
        self.kwargs = kwargs

    def forward(self, x):
        assert isinstance(x, torch.Tensor)
        return einops.rearrange(x, self.rearrange_expr, **self.kwargs)

In [10]:
import torch
import torch.nn as nn
import einops

class ExampleModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Rearrange expression for 5-dimensional tensor
        self.rearrange_layer = EinOpsRearrange('a b (c d) e -> a (b d) (e c)')

    def forward(self, x):
        x = self.rearrange_layer(x)
        return x

# Example tensor with 5 dimensions
x = torch.randn(2, 3, 4, 5, 6)  # Shape: [a, b, c, d, e]

# Initialize and apply EinOpsRearrange
model = ExampleModel()
output = model(x)

print("Original shape:", x.shape)
print("Rearranged shape:", output.shape)


EinopsError:  Error while processing rearrange-reduction pattern "a b (c d) e -> a (b d) (e c)".
 Input tensor shape: torch.Size([2, 3, 4, 5, 6]). Additional info: {}.
 Wrong shape: expected 4 dims. Received 5-dim tensor.

In [11]:
from einops import rearrange, reduce, repeat

In [18]:
# rearrange elements according to the pattern
input_tensor = torch.randn(2, 3, 4)
output_tensor = rearrange(input_tensor, 't b c -> b c t')
print(input_tensor.shape)
print(output_tensor.shape)

torch.Size([2, 3, 4])
torch.Size([3, 4, 2])


In [19]:
# combine rearrangement and reduction
input_tensor = torch.randn(2, 3, 4, 8)
output_tensor = reduce(input_tensor, 'b c (h h2) (w w2) -> b h w c', 'mean', h2=2, w2=2)
# here b is 2, c is 3, h is 2, w is 4
print(input_tensor.shape)
print(output_tensor.shape)

torch.Size([2, 3, 4, 8])
torch.Size([2, 2, 4, 3])


In [20]:
# copy along a new axis
input_tensor = torch.randn(2, 3)
output_tensor = repeat(input_tensor, 'h w -> h w c', c=4)
print(input_tensor.shape)
print(output_tensor.shape)
print(output_tensor)

torch.Size([2, 3])
torch.Size([2, 3, 4])
tensor([[[ 0.3804,  0.3804,  0.3804,  0.3804],
         [ 0.0922,  0.0922,  0.0922,  0.0922],
         [ 0.5896,  0.5896,  0.5896,  0.5896]],

        [[-0.3321, -0.3321, -0.3321, -0.3321],
         [-0.2348, -0.2348, -0.2348, -0.2348],
         [-0.7903, -0.7903, -0.7903, -0.7903]]])


# **SelectEOSAndProject**

In [24]:
import torch
import torch.nn as nn

class SelectEOSAndProject(nn.Module):
    def __init__(self, proj: nn.Module) -> None:
        super().__init__()
        self.proj = proj

    def forward(self, x, seq_len):
        assert x.ndim == 3
        # x is of shape B x L x D
        # take features from the EOS embedding (EOS token is the highest number in each sequence)
        print("torch.arange(x.shape[0]):",torch.arange(x.shape[0]))
        print("seq_len:",seq_len)
        print('x.shape',x.shape)
        print('Before, x = ',x)
        x = x[torch.arange(x.shape[0]), seq_len]
        print('After, x=',x)
        x = self.proj(x)
        return x

# Define a batch of sequences with random values
input_tensor = torch.tensor([
    [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]],  # Sequence 1
    [[10.0, 11.0, 12.0], [13.0, 14.0, 15.0], [16.0, 17.0, 18.0]]  # Sequence 2
])  # Shape: [batch_size, sequence_length, embedding_dim]

# Indices of EOS tokens for each sequence in the batch
seq_len = torch.tensor([2, 1])  # EOS token indices

# Define the projection layer
proj_layer = nn.Linear(3, 2)  # Projects from embedding_dim 3 to some other dimension (e.g., 2)

# Define the SelectEOSAndProject module
select_eos_and_project = SelectEOSAndProject(proj=proj_layer)

# Forward pass
output_tensor = select_eos_and_project(input_tensor, seq_len)

# Print shapes and output
print(f"Input Tensor Shape: {input_tensor.shape}")
print(f"Output Tensor Shape: {output_tensor.shape}")
print(f"Output Tensor: {output_tensor}")


torch.arange(x.shape[0]): tensor([0, 1])
seq_len: tensor([2, 1])
x.shape torch.Size([2, 3, 3])
Before, x =  tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.]],

        [[10., 11., 12.],
         [13., 14., 15.],
         [16., 17., 18.]]])
After, x= tensor([[ 7.,  8.,  9.],
        [13., 14., 15.]])
Input Tensor Shape: torch.Size([2, 3, 3])
Output Tensor Shape: torch.Size([2, 2])
Output Tensor: tensor([[ 7.0366,  2.0731],
        [12.1823,  2.7786]], grad_fn=<AddmmBackward0>)
